In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Conv1D, MaxPooling1D, Flatten, Add, ReLU, LSTM, Reshape, Concatenate, Activation
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE, RandomOversampler

import pickle
import os
from contextlib import redirect_stdout

In [ ]:
#import MIT data
df_mitbih_test = pd.read_csv('data/original/mitbih_test.csv', header = None)

X_train = pd.read_csv('data/processed/X_train.csv')
y_train = pd.read_csv('data/processed/y_train.csv')
y_train = y_train['187']

X_train_sm = pd.read_csv('data/processed/X_train_sm.csv')
y_train_sm = pd.read_csv('data/processed/y_train_sm.csv')
y_train_sm = y_train_sm['187']

X_val = pd.read_csv('data/processed/X_val.csv')
y_val = pd.read_csv('data/processed/y_val.csv')
y_val = y_val['187']

X_test = df_mitbih_test.drop(187, axis = 1)
y_test = df_mitbih_test[187]


# Reshape the data for 1D CNN
X_train_sm_cnn = np.expand_dims(X_train_sm, axis=2)
X_val_cnn = np.expand_dims(X_val, axis=2)
X_test_cnn = np.expand_dims(X_test, axis=2) 

display(X_train_sm_cnn.shape)
display(X_val_cnn.shape)
display(X_test_cnn.shape)

In [ ]:
#Function to plot and save loss and accuracy over epochs from history
def plot_training_history(history, save_dir, prefix): 
    hist = history.history
    metrics = [m for m in hist.keys() if not m.startswith('val_')]  

    # Create the output folder if it does not exist
    os.makedirs(save_dir, exist_ok=True)

    for m in metrics:
        plt.figure()
        plt.plot(hist[m], label=f'Train {m}')
        if f'val_{m}' in hist:
            plt.plot(hist[f'val_{m}'], label=f'Val {m}')
        plt.xlabel('Epoch')
        plt.ylabel(m)
        plt.title(f'{m} over epochs')
        plt.legend()
        plt.grid(True)

        # Construct filename with prefix and filepath with directory and filename
        filename = f"{prefix}_{m}.png"
        filepath = os.path.join(save_dir, filename)

        # Save the figure
        plt.savefig(filepath, format='png', dpi=300, bbox_inches='tight')
        print(f"Saved: {filepath}")
        plt.show()

In [ ]:
#Used LSTMs

#LSTM1, 2 layers
lstm1 = Sequential()

# First LSTM layer
lstm1.add(LSTM(units=32, return_sequences=True, input_shape=(187, 1)))

# Second LSTM layer
lstm1.add(LSTM(units=32, return_sequences=False))

# Fully connected layer
lstm1.add(Dense(32, activation='relu'))

# Output layer with softmax activation for classification
lstm1.add(Dense(5, activation='softmax')) 



#LSTM2, 6 layers
lstm2 = Sequential()

# First LSTM layer
lstm2.add(LSTM(units=32, return_sequences=True, input_shape=(187, 1)))

# Second LSTM layer
lstm2.add(LSTM(units=32, return_sequences=True))

# Third LSTM layer
lstm2.add(LSTM(units=32, return_sequences=True))

# Fourth LSTM layer
lstm2.add(LSTM(units=32, return_sequences=True))

# Fifth LSTM layer
lstm2.add(LSTM(units=32, return_sequences=True))

# Sixth LSTM layer
lstm2.add(LSTM(units=32, return_sequences=False))

# Fully connected layer
lstm2.add(Dense(32, activation='relu'))

# Output layer with softmax activation for classification
lstm2.add(Dense(5, activation='softmax')) 


In [ ]:
lstm1.summary()

lstm2.summary()

In [ ]:
#lr reduction
lrredpl = ReduceLROnPlateau(
    monitor='val_loss',  
    factor=0.5,         
    patience=10,         
    min_lr=1e-7,        
    verbose=1, 
    min_delta=0.001 
)

# Learning rate with exponential decay
initial_learning_rate = 1e-3
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96)


#Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',       
    patience=20,               
    restore_best_weights=True, 
    min_delta=0.001  
)


#Compile when lr exp decay
lstm1.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

# Define where and how to save the best model
checkpoint = ModelCheckpoint(
    filepath='../LSTM_output/lstm1_2layers_sm_lr_bs_epoch_{epoch:02d}_valloss_{val_loss:.4f}.keras',   # file path (can be .keras or .h5)
    monitor='val_loss',       
    mode='min',                    
    save_best_only=False,           
    verbose=1                      
)

In [ ]:
#Train
history = lstm1.fit(
    X_train_sm_cnn,
    y_train_sm,
    epochs=200,
    batch_size=512,
    validation_data=(X_val_cnn, y_val),
    callbacks=[checkpoint, early_stop] 
)

In [ ]:
plot_training_history(history, save_dir="drive/MyDrive/COLAB/final/lstm2_sm", prefix="lstm2_sm_lrexpdec_earlystop_bs512")

In [ ]:
#Save training history
with open("../LSTM_output/lstm1_2layers_sm_lr_bs_epoch__valloss_.pkl", "wb") as f: #change for model
    pickle.dump(history.history, f)

    
best_model = load_model('../LSTM_output/lstm1_2layers_sm_lr_bs_epoch__valloss_.keras') #change for model


#prediction of test data
test_pred = best_model.predict(X_test_cnn)
y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis=1)


#classification report
print(classification_report(y_test_class, y_pred_class, digits=4))


#confusion matrix
print(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions']))


#save results of metrics
with open("../LSTM_output/lstm1_2layers_sm_lr_bs_epoch__valloss_.txt", "w") as file: #change for model
    
    file.write("\nModel: LSTM1\n")#change for model
        
    file.write("\nData augmentation: Smote\n")
    
    file.write("\nConfusion Matrix on test set:\n")
    file.write(str(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions'])))
    
    file.write("\n\nClassification Report on test set:\n")
    file.write(classification_report(y_test_class, y_pred_class, digits=4))